In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:

data_frame = pd.read_csv("vegetable price.csv")
#converting some columns from string to list
def convertto_int(columns):
  r = []
  for x in columns:
    r.append(int(x))
  return r

data_frame['न्यूनतम'] = convertto_int(data_frame['न्यूनतम'].tolist())
data_frame['अधिकतम'] = convertto_int(data_frame['अधिकतम'].tolist())
data_frame['औसत'] = convertto_int(data_frame['औसत'].tolist())
#data_frame.head()

In [3]:
data_frame.drop('अधिकतम', axis=1, inplace=True)



In [4]:
data_frame.drop('न्यूनतम', axis=1, inplace=True)


In [5]:
data_frame.rename(columns={'औसत':'Average'}, inplace=True)


In [6]:
data_frame['cdate'] = pd.to_datetime(data_frame['cdate'])


In [7]:

oneday_vegetable = data_frame[data_frame['cdate'] == '02/25/2018']
wholesell = oneday_vegetable[oneday_vegetable['pricetype'] == 'W']
retell = oneday_vegetable[oneday_vegetable['pricetype'] == 'R']
#plt.bar(wholesell['कृषि उपज'], wholesell['Average'])
wholesell['कृषि उपज']


        

0     गोलभेडा ठूलो(नेपाली)
1             गोलभेडा सानो
2                अालु रातो
3                अालु सेतो
4      प्याज सुकेको भारतीय
              ...         
63              लसुन हरियो
64             हरियो धनिया
65      लसुन सुकेको चाइनिज
66      लसुन सुकेको नेपाली
67               माछा ताजा
Name: कृषि उपज, Length: 68, dtype: object

In [8]:
wholesell['Average'].idxmax()

47

In [9]:
kagati_trend = data_frame[data_frame['कृषि उपज'] == 'कागती']
kagati_trend = kagati_trend.reset_index( drop = True)
kagati_trend.head()

kagati_trend[kagati_trend['cdate'] == '2020-02-01']
kagati_feb = kagati_trend.iloc[1393:, :]
kagati_feb = kagati_feb.reset_index(drop = True)

#kagati_feb

In [10]:
kagati_wholeseller = kagati_feb[kagati_feb['pricetype'] == 'W']
kagati_wholeseller = kagati_wholeseller.reset_index(drop = True)
kagati_wholeseller
kagati_reteller = kagati_feb[kagati_feb['pricetype'] == 'R']
kagati_reteller = kagati_reteller.reset_index(drop = True)
#kagati_wholeseller



In [11]:
#figure2, ax2 = plt.subplots()
#plt.plot(kagati_wholeseller['cdate'], kagati_wholeseller['Average'])


# Fearure selection and extraction

In [12]:
vegetable = set(data_frame['कृषि उपज'].values)

In [13]:
feature_df = []
for v in vegetable:
    for p_type in ['W', 'R']:
        df_v = data_frame[(data_frame['कृषि उपज'] == v) & (data_frame['pricetype'] == p_type)]
        df_v = df_v.sort_values(by = ['cdate'], ascending = False)
        if p_type == 'W':
            df_v['type'] = 1
        else:
            df_v['type'] = 0
        df_v['oneday_before'] = df_v.shift(periods = -1)['Average']
        df_v['twoday_before'] = df_v.shift(periods = -2)['Average']
        feature_df.append(df_v)

feature_df = pd.concat(feature_df)
feature_df.dropna(inplace = True)

        

In [14]:
feature_df = feature_df.sample(frac=1, random_state=1)
feature_df.head()
X = feature_df[['type', 'oneday_before', 'twoday_before']].values
X = np.insert(X,0,1,axis=1)
X

array([[  1.,   1.,  75.,  75.],
       [  1.,   1., 108., 105.],
       [  1.,   1.,  42.,  42.],
       ...,
       [  1.,   1., 105., 115.],
       [  1.,   1.,  48.,  58.],
       [  1.,   0.,  85.,  85.]])

In [15]:

Y = feature_df['Average'].values

In [16]:
total = len(X)
train = int(0.6 * total)
val = int(0.2*total)


X_train = X[:train]
Y_train = Y[:train].reshape(-1,1)

X_val = X[train:train+val]
Y_val = Y[train:train+val].reshape(-1,1)

X_test = X[ train+val: ]
Y_test = Y[train+val:].reshape(-1,1)
len(X_train)
len(Y_train)


59293

In [17]:
norm_xtrain = X_train[:, 2:]
norm_xval = X_val[:,2:]
norm_xtest = X_test[:,2:]
norm_xtrain

#since two features have large value so we must normalize for that we have to take min and max value from training set
min_value = norm_xtrain.min()
max_value = norm_xtrain.max()

def normalize_function(min_val, max_val,to_normalize):
    norm = []
    for i in to_normalize:
        for j in i:
            norm.append((j-min_val)/(max_val-min_val))
    return norm

norm_xtrain = np.asarray(normalize_function(min_value, max_value, norm_xtrain)).reshape(-1,2)
norm_xtest = np.asarray(normalize_function(min_value,max_value,norm_xtest)).reshape(-1,2)
norm_xval = np.asarray(normalize_function(min_value,max_value,norm_xval)).reshape(-1,2)

    
    
    




In [18]:
X_train = np.concatenate((X_train[:, :2], norm_xtrain), axis = 1)
X_val = np.concatenate((X_val[:,:2], norm_xval), axis = 1)
X_test = np.concatenate((X_test[:,:2],norm_xtest), axis = 1)

In [19]:
def mse(h,y):
    return np.average((h-y)**2)



In [26]:

def model_train(x,y,param,a,b):
    np.random.seed(42)
    w =np.random.rand(X_train.shape[1]).reshape(-1,1)
    
    for i in range(0, param['no_of_iteration']):
        pred = np.matmul(X_train, w)
        train_loss = mse(pred , Y_train)
        #print(i,train_loss)
        gradient = np.matmul(np.transpose( pred - Y_train), X_train)/X_train.shape[0]
        gradient = np.transpose(gradient)
        gradient
        w = w - (param['learning_rate']*gradient)
    return w
    
    pred_val = np.matmul(a,w)
    val_error = mse(pred_val,Y_val)
    print(val_error)
    print(np.average(val_error))
    print(param, val_error)
        
   #validation lost is calculated to check if model work fine, it is used after training that is it is not used to update weight
    
    
        

In [21]:
parama = {'iteration' :[500, 1000, 800],
         'learning_rates' : [0.8, 1, 0.01]}
        
 
for i in parama['iteration']:
    for j in parama['learning_rates']:
       param = {
           'no_of_iteration' : i,
           'learning_rate' : j
       }
        
       model_train(X_train, Y_train,param, X_val, Y_val)
       


309.93970341924756
309.93970341924756
{'no_of_iteration': 500, 'learning_rate': 0.8} 309.93970341924756
220.15319337741593
220.15319337741593
{'no_of_iteration': 500, 'learning_rate': 1} 220.15319337741593
6008.314433398116
6008.314433398116
{'no_of_iteration': 500, 'learning_rate': 0.01} 6008.314433398116
160.038986864703
160.038986864703
{'no_of_iteration': 1000, 'learning_rate': 0.8} 160.038986864703
154.87204854859473
154.87204854859473
{'no_of_iteration': 1000, 'learning_rate': 1} 154.87204854859473
5598.255350198743
5598.255350198743
{'no_of_iteration': 1000, 'learning_rate': 0.01} 5598.255350198743
175.15978595802366
175.15978595802366
{'no_of_iteration': 800, 'learning_rate': 0.8} 175.15978595802366
160.02127701622294
160.02127701622294
{'no_of_iteration': 800, 'learning_rate': 1} 160.02127701622294
5728.94265313703
5728.94265313703
{'no_of_iteration': 800, 'learning_rate': 0.01} 5728.94265313703


In [27]:
# #Test set

n = X_test.shape[0]
k = X_test.shape[1] - 1
parameter = {
    'no_of_iteration':1000,
    'learning_rate':1
}
weight_test = model_train(X_train,Y_train,parameter,X_val, Y_val)
weight_test
pred = np.matmul(X_test, weight_test)
sse = mse(pred, Y_test)
sse
Y_test_mean = np.mean(Y_test)
Y_test_mean
sst = mse(Y_test_mean, Y_test)
r2_1 = sse/(n-k-1)
r2_2 = sst/(n-1)
R_2score = 1 - (r2_1/r2_2)
R2 = 1-(sse/sst)
#R_2score
R2



0.9754860431250907